In [105]:
import cv2
import numpy as np
import mahotas
from skimage import color
from matplotlib import pyplot as plt
from skimage import io, color, filters, util
import matplotlib.pyplot as plt


In [91]:
# organize dataset into a useful structure
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
import os

# feature-descriptor-1: Hu Moments
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

# feature-descriptor-2: Haralick Texture
def fd_haralick(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    # return the result
    return haralick



In [128]:
from skimage import measure, io, img_as_bool
import numpy as np
import glob
from cv2 import Sobel
from sklearn import svm, metrics, neighbors

files = glob.glob("Dataset/Motor/Train/*-i/*.jpg")
train_shape_feature = np.zeros((30,3))
for k in range(len(files)):
    BW1 = io.imread(files[k])
    G = util.img_as_ubyte(color.rgb2gray(BW1))
    T = filters.threshold_otsu(G)
    S = util.img_as_float(G < T)
    sf = measure.regionprops(S.astype(int))
    shape_feature = [max(s.moments_hu[0] for s in sf),
    max(s.moments_hu[1] for s in sf),
    max(s.moments_hu[2] for s in sf),]
    train_shape_feature[k,:] = shape_feature
np.savetxt('train_shape_feature.csv', train_shape_feature, delimiter=',', fmt='%f')

files = glob.glob("Dataset/Motor/Test/*-i/*.jp*")
test_shape_feature = np.zeros((15,3))
for k in range(len(files)):
    BW1 = io.imread(files[k])
    G = util.img_as_ubyte(color.rgb2gray(BW1))
    T = filters.threshold_otsu(G)
    S = util.img_as_float(G < T)
#     img = np.float32(BW1) / 255.0 
#     gx = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=1)
#     gy = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=1)
#     mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
    sf = measure.regionprops(S.astype(int))
    shape_feature = [max(s.moments_hu[0] for s in sf),
    max(s.moments_hu[1] for s in sf),
    max(s.moments_hu[2] for s in sf),]
    test_shape_feature[k,:] = shape_feature
np.savetxt('test_shape_feature.csv', test_shape_feature, delimiter=',', fmt='%f')

def accuracy(actual,predicted):
    """This function evaluates the performance of a classification model
    by calculating the Accuracy
    actual = Column matrix with actual class labels of the training
    examples
    predicted = Column matrix with predicted class labels by the
    classification model
    Output: Row matrix with all the performance measures"""
    p = np.count_nonzero(actual == 1)
    n = np.count_nonzero(actual != 1)
    N = p + n
    tp= sum((int(actual[i] == 1 and predicted[i] == 1) for i in range(len(actual))))
    tn = sum((int(actual[i] != 1 and predicted[i] != 1) for i in range(len(actual))))
    fp = n - tn
    fn = p - tp
    accuracy = (tp + tn) / N
    return accuracy

train_label = np.ones((30,1))
train_label[10:20] = 2
train_label[20:] = 3

test_label = np.ones((15,1))
test_label[5:10] = 2
test_label[10:] = 3

train_shape_feature=np.genfromtxt('train_shape_feature.csv',delimiter=',')
test_shape_feature=np.genfromtxt('test_shape_feature.csv',delimiter=',')
 
clf = neighbors.KNeighborsClassifier(3)
clf.fit(train_shape_feature, train_label)
svm_label = clf.predict(test_shape_feature)
cm_svm = metrics.confusion_matrix(svm_label, test_label)
print(cm_svm)

accuracy = accuracy(test_label,svm_label) * 100
print("Akurasi: %f%%" % (accuracy))

15
[[5 3 3]
 [0 2 1]
 [0 0 1]]
Akurasi: 60.000000%


c:\users\jeremia delviero\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:76: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [124]:
from skimage import measure, io, img_as_bool
import numpy as np
import glob
from cv2 import Sobel
from sklearn import svm, metrics, neighbors

files = glob.glob("Dataset/Motor/Train/*-i/*.jpg")
train_texture_feature = np.zeros((30,13))
for k in range(len(files)):
    BW1 = io.imread(files[k])

    # extract haralick texture from the image
    features = fd_haralick(BW1)

    train_texture_feature[k,:] = features
np.savetxt('train_texture_feature.csv', train_texture_feature, delimiter=',', fmt='%f')

files = glob.glob("Dataset/Motor/Test/*-i/*.jp*")
test_texture_feature = np.zeros((15,13))
for k in range(len(files)):
    BW1 = io.imread(files[k])

    # extract haralick texture from the image
    features = fd_haralick(BW1)

    test_texture_feature[k,:] = features
np.savetxt('test_texture_feature.csv', test_texture_feature, delimiter=',', fmt='%f')

train_label = np.ones((30,1))
train_label[10:20] = 2
train_label[20:] = 3

test_label = np.ones((15,1))
test_label[5:10] = 2
test_label[10:] = 3

train_texture_feature=np.genfromtxt('train_texture_feature.csv',delimiter=',')
test_texture_feature=np.genfromtxt('test_texture_feature.csv',delimiter=',')
 
clf = neighbors.KNeighborsClassifier(3)
clf.fit(train_texture_feature, train_label)
svm_label = clf.predict(test_texture_feature)
cm_svm = metrics.confusion_matrix(svm_label, test_label)
print(cm_svm)

def accuracy(actual,predicted):
    """This function evaluates the performance of a classification model
    by calculating the Accuracy
    actual = Column matrix with actual class labels of the training
    examples
    predicted = Column matrix with predicted class labels by the
    classification model
    Output: Row matrix with all the performance measures"""
    p = np.count_nonzero(actual == 1)
    n = np.count_nonzero(actual != 1)
    N = p + n
    tp= sum((int(actual[i] == 1 and predicted[i] == 1) for i in range(len(actual))))
    tn = sum((int(actual[i] != 1 and predicted[i] != 1) for i in range(len(actual))))
    fp = n - tn
    fn = p - tp
    accuracy = (tp + tn) / N
    return accuracy

accuracy_1 = accuracy(test_label,svm_label) * 100
print("Akurasi: %f%%" % (accuracy_1))

[[2 2 0]
 [2 2 2]
 [1 1 3]]
Akurasi: 66.666667%


c:\users\jeremia delviero\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:41: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
